<img src="assets/logo.jpg" alt="logo" width="150" height="80" style="float: left; margin-right: 10px;">

<br><br><br>

<h1><center>Final Deliverable - Artificial Intelligence (Machine Learning)</center></h1>
<br><br><br>

<center><img src="./assets/ai.png" alt="map" width="500" height="400"></center>
<br><br><br>

Delphin Bonheur NGOGA - Nell PATOU PARVEDY - Clément MARQUES - Joel Stephane YANKAM NGUEGUIM


## 1. Introduction
HumanForYou is a pharmaceutical company based in India, employing around 4,000 people. Each year, the company faces a high employee turnover rate of approximately 15%, which causes significant challenges: the projects on which departing employees were working fall behind schedule, damaging the company’s reputation with customers and partners. 
Furthermore, this turnover requires maintaining a large human resources team to recruit, train, and onboard new employees, leading to increased costs and productivity loss.


In this context, management has requested an in-depth analysis of the available data to identify the main factors influencing turnover. The goal is to propose predictive models to guide actions that will motivate employees to stay. This project relies on analyzing several anonymized datasets containing demographic, professional, satisfaction, and performance information about employees.


Alongside technical and operational challenges, adopting an ethical approach is essential. The use of artificial intelligence and predictive analytics in human resources raises sensitive issues related to personal data protection, prevention of discrimination, transparency of models, and preservation of individual autonomy. Respecting these principles is crucial to building trust and ensuring that the solution does not negatively impact employees’ quality of work life or fundamental rights.


## 2. Ethics
This study is therefore part of an approach that combines analytical performance with ethical requirements, particularly aligned with the European Commission’s Assessment List for Trustworthy Artificial Intelligence (ALTAI) recommendations.


### 2.1 Data supplied
The data supplied by HumanForYou comprises several anonymized CSV files, each providing distinct but complementary information for the turnover analysis:

<br>
•	General Human Resources Data (general_data.csv)

This file contains key demographic and professional attributes for each employee identified by a unique EmployeeID for the year 2015. This comprehensive dataset forms the backbone for understanding employee profiles.

<br>
•	Latest Manager Assessment (manager_survey_data.csv)

This smaller file contains each employee’s latest evaluation by their manager as of February 2015. It includes ratings for job involvement (from low to very high) and overall performance (from low to beyond expectations). These assessments provide insights into individual engagement and productivity levels as viewed by direct supervisors.

<br>
•	Workplace Quality of Life Survey (employee_survey_data.csv)

Collected in June 2015, this survey gathers employee feedback on three dimensions of job satisfaction: satisfaction with the work environment, the job itself, and work-life balance. This dataset helps capture subjective factors influencing morale and retention.

<br>
•	Working Hours Data (in_out_time.zip)

Two files within this archive record employee arrival and departure times daily throughout 2015. This time clock data allows analysis of working patterns and potential correlations between attendance behavior and attrition, offering a fine-grained temporal perspective on employee engagement.
Together, these anonymized datasets provide a multi-dimensional view of employees, combining objective HR data, managerial evaluations, subjective satisfaction measures, and actual attendance records in 2015, and the turnover outcome relates to departures in 2016. This rich data environment enables a thorough exploration of factors impacting turnover at HumanForYou.

<br><br>
### 2.2 Analysis According to ALTAI Ethical Requirements

The approach adopted for this project is grounded in the principles of trustworthy artificial intelligence, as outlined by the European Commission’s Assessment List for Trustworthy Artificial Intelligence (ALTAI). These principles include respect for human autonomy, technical robustness and security, data privacy and governance, transparency, fairness and non-discrimination, societal well-being, and accountability. 

<center><img src="assets/ethic.png" alt="map" width="400" height="400"></center>

<br><br>
### 2.3 Analysis According to ALTAI Ethical Requirements

The tool supports the actionability the key requirements outlined by the Ethics Guidelines for Trustworthy Artificial Intelligence (AI), presented by  the High-Level Expert Group on AI (AI HLEG) presented to the European Commission, in April 2019. The Ethics Guidelines introduced the concept of Trustworthy AI, based on seven key requirements:

-	Human Autonomy and Oversight

-	Technical Robustness and Safety

-	Data Privacy and Governance

-	Transparency

-	Diversity, Non-Discrimination and Fairness

-	Societal and Environmental Well-being

-	Accountability





## 3. Data loading and preparation

### 3.1 Data import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer

### 3.2 Loading data

Here we display the first 5 rows of each file:

In [ ]:
# Load dataset
general_data = pd.read_csv('general_data.csv')

# Display the first lines of the dataset
general_data.head()

In [ ]:
employee_survey_data = pd.read_csv('employee_survey_data.csv')
employee_survey_data.head()


In [ ]:
manager_survey_data = pd.read_csv('manager_survey_data.csv')
manager_survey_data.head()

In [ ]:
intime = pd.read_csv('in_time.csv')
intime.head()

In [ ]:
outtime = pd.read_csv('out_time.csv')
outtime.head()

### Merge Datasets

In [ ]:
# merge datasets
data = pd.merge(general_data, manager_survey_data, on='EmployeeID')
data = pd.merge(data, employee_survey_data, on='EmployeeID')

# Display basic information
print(f"  Merged data: {len(data)} employees")
print(f"   Columns: {len(data.columns)}")
print("First 5 columns:", list(data.columns[:5]))

## Data preparation
### 1. handling missing values

In [ ]:

# 1. Sélection STRICTE des colonnes numériques
general_num = general_data.select_dtypes(include="number")

# 2. Comptage des valeurs manquantes (NaN) uniquement sur les colonnes numériques
missing_per_column = general_num.isna().sum()

print("Valeurs manquantes par colonne numérique :")
print(missing_per_column)

# 3. Nombre total de valeurs manquantes numériques
total_missing = missing_per_column.sum()
print(f"\nNombre total de valeurs manquantes (numériques) : {total_missing}")

# 4. Imputation par la médiane
imputer = SimpleImputer(strategy="median")
general_num_imputed_array = imputer.fit_transform(general_num)

# 5. Reconstruction du DataFrame en conservant index et colonnes
general_num_imputed = pd.DataFrame(
    general_num_imputed_array,
    columns=general_num.columns,
    index=general_data.index
)

# 6. Vérification post-imputation
print("\nValeurs manquantes après imputation :")
print(general_num_imputed.isna().sum())

In [ ]:
# 1. Sélection STRICTE des colonnes numériques
employee_num = employee_survey_data.select_dtypes(include="number")

# 2. Comptage des valeurs manquantes (NaN) uniquement sur les colonnes numériques
missing_per_column = employee_num.isna().sum()

print("Valeurs manquantes par colonne numérique :")
print(missing_per_column)

# 3. Nombre total de valeurs manquantes numériques
total_missing = missing_per_column.sum()
print(f"\nNombre total de valeurs manquantes (numériques) : {total_missing}")

# 4. Imputation par la médiane
imputer = SimpleImputer(strategy="median")
employee_num_imputed_array = imputer.fit_transform(employee_num)

# 5. Reconstruction du DataFrame en conservant index et colonnes
employee_num_imputed = pd.DataFrame(
    employee_num_imputed_array,
    columns=employee_num.columns,
    index=employee_survey_data.index
)

# 6. Vérification post-imputation
print("\nValeurs manquantes après imputation :")
print(employee_num_imputed.isna().sum())


In [ ]:
# 1. Sélection STRICTE des colonnes numériques
manager_num = manager_survey_data.select_dtypes(include="number")

# 2. Comptage des valeurs manquantes (NaN) uniquement sur les colonnes numériques
missing_per_column = manager_num.isna().sum()

print("Valeurs manquantes par colonne numérique :")
print(missing_per_column)

# 3. Nombre total de valeurs manquantes numériques
total_missing = missing_per_column.sum()
print(f"\nNombre total de valeurs manquantes (numériques) : {total_missing}")

# 4. Imputation par la médiane
imputer = SimpleImputer(strategy="median")
manager_num_imputed_array = imputer.fit_transform(manager_num)

# 5. Reconstruction du DataFrame en conservant index et colonnes
manager_num_imputed = pd.DataFrame(
    manager_num_imputed_array,
    columns=manager_num.columns,
    index=manager_survey_data.index
)

# 6. Vérification post-imputation
print("\nValeurs manquantes après imputation :")
print(manager_num_imputed.isna().sum())


In [ ]:
# Remove columns with more than 50% missing values
threshold = 0.5
intime_cleaned = intime.loc[:, intime.isnull().mean() < threshold]

print(f"Columns before: {intime.shape[1]}")
print(f"Columns after: {intime_cleaned.shape[1]}")

# Display missing values for remaining columns
print("\nMissing values in remaining columns:")
print(intime_cleaned.isnull().sum())

In [ ]:
# Remove columns with more than 50% missing values
threshold = 0.5
outtime_cleaned = outtime.loc[:, outtime.isnull().mean() < threshold]

print(f"Columns before: {outtime.shape[1]}")
print(f"Columns after: {outtime_cleaned.shape[1]}")

# Display missing values for remaining columns
print("\nMissing values in remaining columns:")
print(outtime_cleaned.isnull().sum())

### 2. Encoding categorical variables

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 1. Select category columns
general_cat = general_data.select_dtypes(include=[object])

# 2. Encoder (dense array)
cat_encoder = OneHotEncoder(sparse_output=False)  # ou sparse=False si version plus ancienne

# 3. Fit and Transform
general_cat_encoded_array = cat_encoder.fit_transform(general_cat)

# 4. Convertir en entiers (0/1 au lieu de 0.0/1.0)
general_cat_encoded_array = general_cat_encoded_array.astype(int)

# 5. Get the Feature Names
feature_names = cat_encoder.get_feature_names_out(general_cat.columns)

# 6. DataFrame final
general_cat_encoded = pd.DataFrame(general_cat_encoded_array, columns=feature_names)

general_cat_encoded.head()

# Créer la nouvelle colonne Attrition
general_cat_encoded["Attrition"] = general_cat_encoded["Attrition_Yes"]
general_cat_encoded["Gender"] = general_cat_encoded["Gender_Female"]

# Optionnel : supprimer les colonnes one-hot d'origine
general_cat_encoded.drop(
    columns=["Attrition_No", "Attrition_Yes", "Over18_Y", "Gender_Female","Gender_Male" ], #on vire Over18 car déductible depuis la variable age
    inplace=True
)

general_cat_encoded.head()

### 3. Data normalization

<center><img src="scale.png" alt="map" width="600" height="200"></center>

In [ ]:
# Standardizing numeric columns
scaler = StandardScaler()
general_num_scaled = pd.DataFrame(scaler.fit_transform(general_num_imputed), columns=general_num_imputed.columns)

# Verification of standardization
general_num_scaled.head()


<center><img src="assets/standardization.png" alt="map" width="650" height="250"></center>
<br><br><br>

In [ ]:
scaler = StandardScaler()
employee_num_scaled = pd.DataFrame(scaler.fit_transform(employee_num_imputed), columns=employee_num_imputed.columns)

employee_num_scaled.head()

In [ ]:
scaler = StandardScaler()
manager_num_scaled = pd.DataFrame(scaler.fit_transform(manager_num_imputed), columns=manager_num_imputed.columns)

manager_num_scaled.head()

At this point, we loaded and prepared our datasets for analysis. We have imputed missing values, encoded categorical variables and normalized numerical variables. The data is now ready for the modeling and classification phase.

## Data Exploration and Visualization

### 1. Analyzing descriptive statistics

Before diving into data visualization, let's take a look at descriptive statistics to get an initial idea of variable distributions.

In [ ]:
general_data.describe()


In [ ]:
employee_survey_data.describe()


In [ ]:
manager_survey_data.describe()


We can see that the dataset is complete for most variables (4,410 observations) but that the temporal variables (intime/outtime) are unusable. Outliers found, for example, in the MonthlyIncome column

### 2. Visualizing Numerical Variables
We'll start by visualizing the distributions of numerical variables to better understand their behavior.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

general_num_scaled.hist(bins=50, figsize=(20, 15))
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

employee_num_scaled.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

manager_num_scaled.hist(bins=50, figsize=(20, 15))
plt.show()

### 3. Correlation matrix
To explore the relationships between numerical variables, we use a correlation matrix and a heatmap.

In [ ]:
non_zero_std_cols = general_num_scaled.columns[general_num_scaled.std() != 0]
general_num_scaled_clean = general_num_scaled[non_zero_std_cols]

employee_num_scaled.drop(
    columns=["EmployeeID" ],
    inplace=True
)

manager_num_scaled.drop(
    columns=["EmployeeID" ],
    inplace=True
)

# 2. Concatenate numerical and categorical features
X = pd.concat(
    [general_num_scaled_clean, general_cat_encoded,employee_num_scaled, manager_num_scaled],
    axis=1
)

# 3. Correlation matrix (NUMERICAL FEATURES ONLY)
corr_matrix = X.corr()

# 4. Plot correlation heatmap
plt.figure(figsize=(40, 16))
sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    fmt=".2f",
    square=True
)
plt.title('Matrice de Corrélation des Variables Numériques')
plt.tight_layout()
plt.show()

### 4. Visualizing relationships between variables

We're going to create a few scatter plots to visualize the relationships between certain key variables.

In [ ]:
attributes = [
    "Age",
    "DistanceFromHome", 
    "Education",
    "MonthlyIncome",
    "NumCompaniesWorked",
    "PercentSalaryHike",
    "TotalWorkingYears",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "YearsWithCurrManager"
]

scatter_matrix = pd.plotting.scatter_matrix(
    general_data[attributes], 
    figsize=(15, 15),
    alpha=0.5,
    diagonal='hist' 
)
plt.suptitle('Scatter Matrix - General Variables', y=1.0)
plt.show()

**Salary analysis**

In [ ]:
attributes = [
    "MonthlyIncome",
    "TotalWorkingYears",
    "YearsAtCompany",
    "Education",
    "JobLevel"
]

scatter_matrix = pd.plotting.scatter_matrix(
    general_data[attributes], 
    figsize=(15, 15),
    alpha=0.5,
    diagonal='hist'  
)
plt.suptitle('Scatter Matrix - Salary Analysis', y=1.0)
plt.show()

**Seniority Analysis**

In [ ]:
attributes = [
    "Age",
    "TotalWorkingYears",
    "YearsAtCompany",
    "YearsWithCurrManager",
    "NumCompaniesWorked"
]

scatter_matrix = pd.plotting.scatter_matrix(
    general_data[attributes], 
    figsize=(15, 15),
    alpha=0.5,
    diagonal='hist'  
)
plt.suptitle('Scatter Matrix -  Seniority Analysis', y=1.0)
plt.show()

**Mobility Analysis**

In [ ]:
attributes = [
    "NumCompaniesWorked",
    "YearsAtCompany",
    "Age",
    "DistanceFromHome",
    "JobLevel"
]

scatter_matrix = pd.plotting.scatter_matrix(
    general_data[attributes], 
    figsize=(15, 15),
    alpha=0.5,
    diagonal='hist'  
)
plt.suptitle('Scatter Matrix -  Mobility Analysis', y=1.0)
plt.show()

**Satisfaction Analysis**

In [ ]:
attributes = [
    "EnvironmentSatisfaction",
    "JobSatisfaction",
    "WorkLifeBalance"
]

scatter_matrix = pd.plotting.scatter_matrix(
    employee_survey_data[attributes], 
    figsize=(10, 10),
    alpha=0.5
)
plt.suptitle('Scatter Matrix -  Satisfaction Analysis', y=1.0)
plt.show()


**Performance Analysis**

In [ ]:
attributes = [
    "JobInvolvement",
    "PerformanceRating"
]

scatter_matrix = pd.plotting.scatter_matrix(
    manager_survey_data[attributes], 
    figsize=(8, 8),
    alpha=0.5
)
plt.suptitle('Scatter Matrix -  Performance Analysis', y=1.0)
plt.show()

Do not include:
<br>
“EmployeeID” = Meaningless identifier
<br>
“EmployeeCount” = Constant (always 1)
<br>
“StandardHours” = Constant (always 8)
<br>
“StockOptionLevel” = Little variance, zero correlations

## Ethics
### 1. Global ethical methodology

The approach adopted for this project is grounded in the principles of trustworthy artificial intelligence, as outlined by the European Commission’s Assessment List for Trustworthy Artificial Intelligence (ALTAI). These principles include respect for human autonomy, technical robustness and security, data privacy and governance, transparency, fairness and non-discrimination, societal well-being, and accountability.

<br>

### 2. Analysis According to ALTAI Ethical Requirements

The tool supports the actionability the key requirements outlined by the Assessment List for Trustworthy Artificial Intelligence, presented by the High-Level Expert Group on AIpresented to the European Commission, in April 2019. Despite our company not being in the EU, this document is an excellent base to build upon, given no Indian equivalent exists. The Ethics Guidelines introduces the concept of Trustworthy AI, based on seven key requirements:


- Human Autonomy and Oversight: AI systems should support human agency and human decision-making, as prescribed by the principle of respect for human autonomy. This includes training individuals to make informed choices, claim their fundamental rights, and ensuring appropriate oversight mechanisms.

- Technical Robustness and Safety: The AI systems must deliver systems that can be trusted and that are resilient. This implies the systems must be implemented with a preventative approach to limit potential unwanted harm and prevent it whenever it could happen.

- Data Privacy and Governance: To ensure trustworthy AI, robust protections for privacy and data quality must be in place throughout the system's lifecycle. This includes respecting data protection regulations (such as DPDP 2023, the Digital Personal Data Protection Act in India), implementing strong mechanisms for data collection, storage, and processing, minimizing data usage where possible, and ensuring high-quality, accurate, and relevant datasets to avoid errors or biases emergingfrom poor data management.

- Transparency: Everything should be traceable, explainable and there should be clear communications about the limitations of the system. This aims to build trust towards AI and to have a good comprehension of its whole way of working.

- Diversity, Non-Discrimination and Fairness: To achieve Trustworthy AI, we must have inclusion and diversity throughout the entire AI system. AI systems may suffer from the inclusion of inadvertent historic bias, incompleteness, and bad governance models. If handled poorly, those biases could lead to unintended direct or indirect prejudice against a certain group of people. We aim to avoid potential harm at any cost.

- Societal and Environmental Well-being: Trustworthy AI should contribute positively to society and the environment, promoting sustainable development and minimizing negative impacts. This involves assessing and addressing potential effects on employment, social cohesion, democracy, and ecological footprint.

- Accountability: Mechanisms must be established to ensure responsibility and accountability for AI systems and their outcomes, both before and after deployment. This includes clear allocation of responsibilities among developers, deployers, and users, auditability of processes and decisions, redress mechanisms for affected individuals, and regular risk assessments to enable traceability and remediation in case of harm.

<center><img src="ethic.png" alt="map" width="400" height="400"></center>

<br>

### 3. Removed features

According to the ALTAI, some data must be avoided or excluded prior to using them to train our model. This includes the following data, present in our dataset:

- Gender: This is potentially sensitive data if it reveals or infers sexual orientation. More broadly, it is a protected characteristic against discrimination. Using it can introduce discriminatory biases, violating ALTAI's non-discrimination principle

- Marital status: This data reveals aspects of private and family life. It can indirectly infer sensitive information.

- Education field: Depending on the field, it can reveal religious or philosophical beliefs or ethnic origin.

- Age: Using it in a predictive model can lead to ageist biases. ALTAI recommends minimising proxies for protected characteristics to avoid discrimination.

- Employee ID: Allows to directly identify an employee.


For efficiency in the handling of data, we also delete some useless fields of the data:

- Over18 is non-variable (always yes)
- Standard hours is always at 8 hours a day
- Employee count is always at 1.

## Classification models
In this section, we will develop and evaluate multiple classification models to identify the best performer for our objectives. We will implement various algorithms, assess their performance using standard metrics, and compare results to select the optimal model.


### Reminder of the main performance metrics

Performance metrics are essential for assessing the effectiveness of classification models. Here's a reminder of the main metrics used:

#### 1. Accuracy
Accuracy is the ratio of the number of correct predictions to the total number of predictions.

$
\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}
$

#### 2. Precision
Precision is the ratio of true positives (TP) to the sum of true positives (TP) and false positives (FP).

$
\text{Precision} = \frac{TP}{TP + FP}
$

#### 3. Recall
Recall is the ratio of true positives (TP) to the sum of true positives (TP) and false negatives (FN).

$
\text{Recall} = \frac{TP}{TP + FN}
$

#### 4. F1-Score
The F1-score is the harmonic mean of precision and recall, providing a balance between the two.

$
\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$


### Confusion Matrix

The confusion matrix is a method of visualizing the performance of a classification model. It displays results in tabular form, with actual and predicted predictions. Its components are :

- **True Positives (TP)**: Number of times the positive class was correctly predicted.
- **False Positive (FP)**: Number of times the negative class was incorrectly predicted as positive.
- **True Negatives (TN)**: Number of times the negative class was correctly predicted.
- **False Negatives (FN)**: Number of times the positive class was incorrectly predicted as negative.

$
\begin{array}{|c|c|c|}
\hline
& \text{Predicted Positive} & \text{Predicted Negative} \\
\hline
\text{True Positive} & \text{TP} & \text{FN} \\
\hline
\text{True Negative} & \text{FP} & \text{TN} \\
\hline
\end{array}
$

<center><img src="assets/metrics.png" alt="map" width="550" height="400"></center>
<br><br><br>

### AUC and ROC curve

#### ROC curve (Receiver Operating Characteristic)

The ROC curve is a graph showing the performance of a classification model at different discrimination thresholds. It plots the True Positive Rate (TPR) against the False Positive Rate (FPR).

- True Positive Rate (TPR)** : This is the recall.

$
TPR = \frac{TP}{TP + FN}
$

- False Positive Rate (FPR)** : This is the ratio of false positives to the sum of true negatives and false positives.

$
FPR = \frac{FP}{FP + TN}
$

#### AUC (Area Under the Curve)

AUC is the area under the ROC curve. It measures the model's ability to distinguish between positive and negative classes. An AUC of 1.0 indicates a perfect model, while an AUC of 0.5 indicates a model that does no better than random choice.

- **AUC interpretation** :
  - **0.9 - 1** : Excellent performance
  - **0.8 - 0.9** : Good performance
  - **0.7 - 0.8** : Acceptable performance
  - **0.6 - 0.7** : Poor performance
  - **0.5 - 0.6** : Very poor performance

ROC curves and AUC scores are valuable tools for comparing the performance of different classification models, particularly in situations where classes are unbalanced.

By using these metrics and tools, you can comprehensively assess the performance of your classification models and choose the one best suited to your problem.

## 1. Target variable and class balance

Before training any model, we analyze the target variable `Attrition`.
This step is critical because class imbalance strongly impacts:
- model behavior (e.g., a model may simply learn to always predict the majority class),
- the interpretation of performance metrics (e.g., accuracy can be misleading when one class dominates),
- the business conclusions we draw from the results (e.g., underestimating the number of employees at risk).

In our dataset, `Attrition` is a binary variable indicating whether an employee has left the organization (`Yes`) or not (`No`).
Understanding the proportion of each class (churned vs. retained employees) will guide the choice of appropriate evaluation metrics and, if necessary, techniques to handle imbalance (such as resampling or class weights).

## Model application

### 0. Preparation of learning and test data


## 1. Feature matrix (X) and target vector (y)


We now construct the feature matrix `X` and the target vector `y` that will be used as inputs for the classification models.

- The **target variable** `y` corresponds to the column `Attrition`, which we convert into a binary numerical format:
  - `No` → `0` (the employee stayed),
  - `Yes` → `1` (the employee left).
- The **feature matrix** `X` contains all the explanatory variables used to predict attrition.
  We remove:
  - the target column `Attrition`, to avoid data leakage,
  - the column `EmployeeID`, which is an identifier and does not carry predictive information.

The corresponding code is:

In [ ]:
X = data.drop(columns=["Attrition", "EmployeeID"])
y = data["Attrition"].astype(str).str.strip().map({"No": 0, "Yes": 1}).astype(int)
X.shape, y.shape


## 2. Train / Test split

- We split the dataset into training and test sets.
Stratification is used to preserve class imbalance proportions.
- We now split the dataset into a **training set** and a **test set** in order to evaluate our models on unseen data.

- The **training set** is used to fit (learn) the model parameters.
- The **test set** is kept aside and only used at the end to assess the generalization performance.

We use a **70/30 split** (`test_size=0.30`), meaning 70% of the data for training and 30% for testing.

To maintain the original class distribution of `Attrition` in both subsets, we enable **stratification** with `stratify=y`.
This is particularly important when the target variable is imbalanced, as it ensures that both the training and test sets reflect the same proportion of positive and negative cases.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


## 3) Show imbalance (counts + % + plot)
- We first inspect the distribution of the target variable `Attrition` to quantify the class imbalance.

- **Counts** tell us how many employees belong to each class (`No` vs. `Yes`).
- **Percentages** indicate the relative frequency of each class in the dataset.
- A **bar plot** visually highlights how skewed the classes are.

The following code computes and displays these elements:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

counts = y.value_counts().sort_index()
pct = (counts / counts.sum() * 100).round(2)

print("Class counts (Attrition):")
print(counts)
print("\nClass percentages (%):")
print(pct)

plt.figure(figsize=(5,4))
plt.bar(["0 (Stay)", "1 (Leave)"], [counts.get(0,0), counts.get(1,0)])
plt.title("Attrition class distribution")
plt.ylabel("Number of employees")
plt.show()


### 4.Build preprocessing (works with mixed types)

Our feature matrix `X` contains both **numerical** and **categorical** variables.
To feed these into most machine learning models, we create a preprocessing pipeline that:

- **Separates columns by type**:
  - `num_cols`: numerical features,
  - `cat_cols`: categorical features.
- **Applies appropriate transformations**:
  - For **numerical features**:
    - Impute missing values with the **median**,
    - Standardize features using `StandardScaler` (zero mean, unit variance).
  - For **categorical features**:
    - Impute missing values with the **most frequent** category,
    - Apply **one-hot encoding** to convert categories into binary indicator variables.

We use a `ColumnTransformer` to apply these pipelines to the correct subsets of columns, and drop any remaining columns.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
], remainder="drop")

preprocess


### 5. Logistic regression

### Logistic regression reminder

Logistic regression is a statistical technique used to model the probability of a binary event (with two possible outcomes) occurring. Unlike linear regression, which predicts a continuous value, logistic regression predicts the probability of an event occurring.

#### Mathematical formulation

Logistic regression uses the logistic or sigmoid function to transform the output of linear regression into a probability.

The logistic function is defined as follows:
$\sigma(z) = \frac{1}{1 + e^{-z}}$

In logistic regression, \( z \) is a linear combination of the features:
$z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_n x_n$

Where:
- $\beta_0$ is the intercept (y-intercept)
- $\beta_1, \beta_2, \ldots, \beta_n$ are the coefficients of the characteristics $x_1, x_2, \ldots, x_n$.

The probability of the event occurring (for example, \( y = 1 \)) is then given by :
$P(y=1|x) = \sigma(z) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_n x_n)}}$

#### Cost function

The cost function used to adjust the logistic regression parameters is the log-likelihood, defined as follows:
$J(\beta) = - \frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{y}^{(i)}) + (1 - y^{(i)}) \log(1 - \hat{y}^{(i)}) \right] $

Where:
- $m$ is the number of samples
- $y^{(i)}$ is the actual value for sample $i$.
- $\hat{y}^{(i)}$ is the predicted probability for sample $i$.

#### Model training

Training the logistic regression model involves finding the $\beta$ parameters that minimize the cost function. This is generally done using the gradient descent algorithm.


In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(class_weight="balanced", max_iter=1000, random_state=42))
])

log_reg.fit(X_train, y_train)


### 6. Perceptron

### A reminder of the Perceptron

The perceptron is one of the simplest and oldest supervised classification algorithms, introduced by Frank Rosenblatt in 1957. It is an elementary processing unit of a neural network, often used for binary classification tasks. The perceptron is based on a linear combination of input features and uses a threshold function to produce a binary output.

#### Mathematical formulation

The perceptron calculates a weighted sum of the input features and applies a threshold function to determine the predicted class.

The perceptron output is defined as follows:
$\hat{y} = \begin{cases}
1 & \text{si } z \geq 0
0 & \text{si } z < 0
\end{cases} $

Where:
$z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \ldots + \beta_n x_n$

Here, $\beta_0$ is the bias (or intercept), and $\beta_1, \beta_2, \ldots, \beta_n$ are the feature weights $x_1, x_2, \ldots, x_n$.

#### Learning algorithm

The perceptron's learning algorithm adjusts weights according to classification errors. For each training sample $(x^{(i)}, y^{(i)})$, where $y^{(i)}$ is the actual class:

1. Calculate the predicted output:
$hat{y}^{(i)} = \begin{cases}
1 & \text{si } z \geq 0
0 & \text{si } z < 0
\end{cases}$

2. Update the weights if the prediction is incorrect:
$\beta_j = \beta_j + \eta (y^{(i)} - \hat{y}^{(i)}) x_j^{(i)}$
Where $\eta$ is the learning rate.

#### Cost function

The perceptron does not use a cost function in the traditional sense such as logistic regression. Weights are updated directly according to classification errors.


In [ ]:

from sklearn.linear_model import Perceptron
perceptron = Pipeline([
    ("preprocess", preprocess),
    ("model", Perceptron(random_state=42))
])
perceptron.fit(X_train, y_train)

**Question:** According to the results obtained, how does Perceptron differ from logistic regression in its approach to classification?

### 6. Random Forest

### Reminder of Random Forests

Random Forests are a powerful and flexible ensemble method used for classification and regression tasks. They combine several decision trees to improve predictive performance and reduce the risk of overlearning.

#### How it works

A random forest is made up of many independent decision trees, each built on a random sample of the training data and using a random subset of the features for each division of the tree. The predictions of all the trees are then combined to produce a single final prediction.

#### Random Forest construction

1. **Bootstrap sampling**: For each tree in the forest, a random sample with bootstrap replacement of the training data is created. This means that some examples may be selected several times, while others may not be selected at all.
2. **Feature Subset Selection**: At each node of each tree, a random subset of the features is selected. The tree selects the best division from this subset of features.
3. **Tree construction**: Decision trees are built to completion without pruning. This allows each tree to capture complex patterns in the data.
4. **Aggregation of Predictions**: For classification, each tree votes for a class, and the class with the most votes is chosen as the final prediction (majority voting). For regression, the average of all tree predictions is used.

#### Advantages and Disadvantages

##### Advantages :
- **Reduces overlearning**: By combining the predictions of several trees, random forests reduce the risk of overlearning compared with individual decision trees.
- **Robustness**: Insensitive to variations in training data. Random forests are less sensitive to fluctuations in training data.
- **Feature Management**: Able to manage a large number of features and determine the most important ones.
- **Missing Data Handling**: Can handle missing values by imputing values based on forest trees.

##### Disadvantages :
- **Complexity and Computing Time** : Random forests require more computation time and memory than individual decision trees, especially when the number of trees is high.
- **Interpretability**: Less interpretable than individual decision trees, due to the combination of many trees.

#### Applications

- **Classification**: Used for classification tasks in diverse fields such as finance, medicine and marketing.
- **Regression**: Prediction of continuous values in contexts such as real estate price forecasting and sales prediction.
- **Feature Selection**: Identification of the most important features for prediction.

Random forests are a powerful tool for improving the predictive performance and robustness of decision models, by combining the strength of multiple decision trees while mitigating their individual weaknesses.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced_subsample",
        random_state=42
    ))
])

rf.fit(X_train, y_train)


## Comparative study of SK-Learn models

## 1. Preparation of learning and test data
We split the dataset into training and test sets using a 70/30 proportion.Stratification on the target ` y `is used to preserve the original class imbalance (same proportion of attrition/no attrition in both sets )


In [ ]:
from sklearn.model_selection import train_test_split

# X, y must already exist:
# X = data.drop(columns=["Attrition","EmployeeID"])
# y = data["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape


## 2. Show imbalance (counts + % + plot)

- We examine the distribution of the target variable `Attrition` (`0 = Stay`, `1 = Leave`)
using class counts, class percentages, and a bar plot.

In [ ]:
counts = y.value_counts().sort_index()
pct = (counts / counts.sum() * 100).round(2)

print(counts)
print(pct)

plt.figure(figsize=(5,4))
plt.bar(["0 (Stay)", "1 (Leave)"], [counts.loc[0], counts.loc[1]])
plt.title("Attrition class distribution")
plt.ylabel("Number of employees")
plt.show()


## 3.Build preprocessing (works with mixed types)
- We build a preprocessing pipeline that handles numerical and categorical features differently:
- numerical: median imputation + standardization
- categorical: most frequent imputation + one‑hot encoding

This pipeline will be combined later with our machine learning models.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X.columns if c not in cat_cols]

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
], remainder="drop")

preprocess


## 4.Define models as full pipelines (preprocess + model)

In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier

models = {
    "LogisticRegression": Pipeline([
        ("preprocess", preprocess),
        ("model", LogisticRegression(max_iter=2000, random_state=42, class_weight="balanced"))
    ]),
    "Perceptron": Pipeline([
        ("preprocess", preprocess),
        ("model", Perceptron(random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("preprocess", preprocess),
        ("model", RandomForestClassifier(
            n_estimators=300,
            random_state=42,
            class_weight="balanced_subsample",
            n_jobs=-1
        ))
    ])
}


## 5. Train models and compute predictions/scores

In [ ]:
def get_score(pipeline, X_):
    """
    Returns continuous scores for ROC/PR:
    - predict_proba[:,1] if available
    - else decision_function if available
    - else None
    """
    if hasattr(pipeline, "predict_proba"):
        return pipeline.predict_proba(X_)[:, 1]
    if hasattr(pipeline, "decision_function"):
        return pipeline.decision_function(X_)
    return None

predictions = {}
scores = {}

for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    predictions[name] = pipe.predict(X_test)
    scores[name] = get_score(pipe, X_test)

list(predictions.keys()), {k: (None if v is None else "ok") for k,v in scores.items()}


## 6.Fit + predict + get continuous scores (ROC/PR)
- We now fit each pipeline on the training data, obtain:
- **hard predictions** (`predict`) for confusion matrices and basic metrics,
- **continuous scores** (probabilities or decision function) for ROC and PR curves.

In [ ]:
def get_score(pipeline, X_):
    """
    Returns continuous scores for ROC/PR:
    - predict_proba[:,1] if available
    - else decision_function if available
    - else None
    """
    if hasattr(pipeline, "predict_proba"):
        return pipeline.predict_proba(X_)[:, 1]
    if hasattr(pipeline, "decision_function"):
        return pipeline.decision_function(X_)
    return None

predictions = {}
scores = {}

for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    predictions[name] = pipe.predict(X_test)
    scores[name] = get_score(pipe, X_test)

list(predictions.keys()), {k: (None if v is None else "ok") for k,v in scores.items()}


## 7. Compute evaluation metrics and build `results` dictionary

We now compute, for each model:
- the **confusion matrix**,
- **Accuracy** and **Balanced Accuracy**,
- **Precision**, **Recall**, and **F1-score** for the positive class (Attrition = 1),
- **ROC AUC** and **PR AUC** (if continuous scores are available).

All metrics are stored in a single `results` dictionary for easy comparison.

In [ ]:
# ================================
# 1) Build `results` (required)
# ================================
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score, balanced_accuracy_score,
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

def build_results(y_test, predictions, scores):
    results = {}
    for name in predictions.keys():
        y_pred = predictions[name]
        y_score = scores.get(name, None)

        cm = confusion_matrix(y_test, y_pred)

        acc = accuracy_score(y_test, y_pred)
        bacc = balanced_accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
        rec  = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
        f1v  = f1_score(y_test, y_pred, pos_label=1, zero_division=0)

        roc_auc = np.nan
        pr_auc = np.nan
        if y_score is not None:
            roc_auc = roc_auc_score(y_test, y_score)
            pr_auc  = average_precision_score(y_test, y_score)

        results[name] = {
            "confusion_matrix": cm,
            "Accuracy": acc,
            "BalancedAcc": bacc,
            "Precision": prec,
            "Recall": rec,
            "F1": f1v,
            "ROC_AUC": roc_auc,
            "PR_AUC": pr_auc
        }
    return results

results = build_results(y_test, predictions, scores)

# ================================
# 2) Plot confusion matrices
# ================================
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(4.6, 4.2))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = [0, 1]
    plt.xticks(tick_marks, ["0", "1"])
    plt.yticks(tick_marks, ["0", "1"])

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, str(cm[i, j]), ha="center", va="center")

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()
    plt.show()

for name, r in results.items():
    plot_confusion_matrix(r["confusion_matrix"], f"Confusion Matrix — {name}")


## 8. Plot ROC and Precision–Recall curves

Using the continuous scores (`scores`) computed earlier, we now:
- plot the **ROC curve** (TPR vs FPR) for each model and display its **AUC**,
- plot the **Precision–Recall curve** for each model and display its **Average Precision (AP)**.

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve

# ROC
plt.figure(figsize=(6,5))
for name in models.keys():
    y_score = scores[name]
    if y_score is None:
        continue
    fpr, tpr, _ = roc_curve(y_test, y_score)
    auc = roc_auc_score(y_test, y_score)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")

plt.plot([0,1],[0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.tight_layout()
plt.show()

# PR
plt.figure(figsize=(6,5))
for name in models.keys():
    y_score = scores[name]
    if y_score is None:
        continue
    prec, rec, _ = precision_recall_curve(y_test, y_score)
    ap = average_precision_score(y_test, y_score)
    plt.plot(rec, prec, label=f"{name} (AP={ap:.3f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.tight_layout()
plt.show()


## 9. Metrics table (model comparison)

We build a single table to compare models on the same test set.
For imbalanced attrition, prioritize:
- Recall (class 1) and F1 (class 1)
- Balanced Accuracy
- PR-AUC (Average Precision)
ROC-AUC is also reported when probability/decision scores are available.


In [ ]:
import pandas as pd
import numpy as np

rows = []

for name, r in results.items():
    rows.append({
        "Model": name,
        "Accuracy": r.get("Accuracy", np.nan),
        "BalancedAccuracy": r.get("BalancedAcc", np.nan),
        "Precision(1)": r.get("Precision", np.nan),
        "Recall(1)": r.get("Recall", np.nan),
        "F1(1)": r.get("F1", np.nan),
        "ROC_AUC": r.get("ROC_AUC", np.nan),
        "PR_AUC": r.get("PR_AUC", np.nan)
    })

metrics_table = pd.DataFrame(rows)

metrics_table


## 10. Cross-Validation

**Principle of Cross-Validation:**
Cross-validation is a statistical method used to estimate the performance of a model on unseen data. It addresses the limitation of a single train-test split, which can be sensitive to how the data is partitioned.

In **k-fold cross-validation**, the dataset is divided into *k* subsets (folds). The model is trained *k* times, each time using *k-1* folds for training and the remaining fold for validation. The final performance metric is the average of the results from all *k* trials. This provides a more robust and reliable estimate of the model"s generalization ability.

In [ ]:
from sklearn.model_selection import cross_val_score

print("Evaluating models using 5-fold cross-validation (Accuracy):")
for name, model in models.items():
    # Using the training set for cross-validation to assess stability
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    print(f"{name:20}: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 11. Feature Importance

**Principle of Feature Importance:**
Feature importance refers to techniques that assign scores to input features based on their contribution to the model"s predictive performance. Understanding feature importance is crucial for:
- **Model Interpretability:** Identifying which factors (e.g., salary, work-life balance, distance from home) most heavily influence model predictions.
- **Feature Selection:** Identifying and potentially removing irrelevant or redundant features to simplify the model and improve performance.
- **Business Insights:** Providing actionable insights to stakeholders about the primary drivers of the target outcome (employee attrition).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# We use the RandomForest model as it provides reliable native feature importance scores
if "RandomForest" in models:
    rf_pipeline = models["RandomForest"]
    # Ensure the model is fitted (it should be from previous steps)
    rf_model = rf_pipeline.named_steps["model"]
    preprocessor = rf_pipeline.named_steps["preprocess"]

    # Get feature names from the preprocessor
    # This requires scikit-learn >= 1.0
    try:
        feature_names = preprocessor.get_feature_names_out()
    except:
        # Fallback if get_feature_names_out is not available
        feature_names = [f"feature_{i}" for i in range(len(rf_model.feature_importances_))]

    # Get importance values
    importances = rf_model.feature_importances_

    # Create a DataFrame for sorting and plotting
    feat_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
    feat_df = feat_df.sort_values(by="Importance", ascending=False).head(15)

    # Plotting
    plt.figure(figsize=(10, 8))
    sns.barplot(x="Importance", y="Feature", data=feat_df, hue="Feature", palette="magma", legend=False)
    plt.title("Top 15 Most Important Features - RandomForest")
    plt.xlabel("Relative Importance Score")
    plt.ylabel("Features")
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("RandomForest model not found in the models dictionary.")
## -----------------------------------------------------------------------------------

## Ethical feature selection

Based on ethical analysis and responsible AI principles, we remove features that may introduce
discrimination, bias, or that have no causal relevance to employee attrition.

### Removed features and justification

- **Gender** → risk of gender discrimination
- **MaritalStatus** → private life attribute, not job-related
- **EducationField** → indirect socio-economic bias
- **Age** → age discrimination risk
- **EmployeeID** → identifier, no predictive value
- **Over18** → constant value, no information
- **StandardHours** → constant value
- **EmployeeCount** → constant value

These features are excluded before model training to ensure fairness, transparency,
and compliance with ethical AI guidelines.


In [ ]:
# ================================
# Ethical feature removal
# ================================

features_to_drop = [
    "Gender",
    "MaritalStatus",
    "EducationField",
    "Age",
    "EmployeeID",
    "Over18",
    "StandardHours",
    "EmployeeCount"
]

X_ethic = data.drop(columns=["Attrition"] + features_to_drop)
y_ethic = data["Attrition"].astype(str).str.strip().map({"No": 0, "Yes": 1}).astype(int)

X_ethic.shape, y_ethic.shape


## Preprocessing after ethical filtering

After removing sensitive and non-informative features, we rebuild the preprocessing pipeline:

- Numerical features → median imputation + standardization
- Categorical features → most frequent imputation + one-hot encoding

This ensures consistency and prevents data leakage.


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

cat_cols = X_ethic.select_dtypes(include=["object"]).columns.tolist()
num_cols = [c for c in X_ethic.columns if c not in cat_cols]

numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_ethic = ColumnTransformer([
    ("num", numeric_pipeline, num_cols),
    ("cat", categorical_pipeline, cat_cols)
])


## Model training with ethically filtered features

We retrain the same models using the ethically filtered dataset:

- Logistic Regression (baseline, interpretable)
- Perceptron (linear classifier)
- Random Forest (non-linear, high performance)

This allows a direct comparison with previous results.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_ethic,
    y_ethic,
    test_size=0.30,
    random_state=42,
    stratify=y_ethic
)


In [ ]:
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier

models_ethic = {
    "LogisticRegression": Pipeline([
        ("preprocess", preprocess_ethic),
        ("model", LogisticRegression(
            max_iter=2000,
            random_state=42,
            class_weight="balanced"
        ))
    ]),
    "Perceptron": Pipeline([
        ("preprocess", preprocess_ethic),
        ("model", Perceptron(random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("preprocess", preprocess_ethic),
        ("model", RandomForestClassifier(
            n_estimators=300,
            random_state=42,
            class_weight="balanced_subsample",
            n_jobs=-1
        ))
    ])
}


In [ ]:
def get_score(pipeline, X_):
    if hasattr(pipeline, "predict_proba"):
        return pipeline.predict_proba(X_)[:, 1]
    if hasattr(pipeline, "decision_function"):
        return pipeline.decision_function(X_)
    return None

predictions_ethic = {}
scores_ethic = {}

for name, pipe in models_ethic.items():
    pipe.fit(X_train, y_train)
    predictions_ethic[name] = pipe.predict(X_test)
    scores_ethic[name] = get_score(pipe, X_test)


## Performance evaluation after ethical feature removal

We evaluate the models using metrics adapted to class imbalance:

- Accuracy
- Balanced Accuracy
- Precision / Recall / F1 (Attrition = 1)
- ROC-AUC
- PR-AUC (Average Precision)

This allows us to assess the impact of ethical constraints on performance.


In [ ]:
# ================================
# 1) Build results_ethic (metrics)
# ================================
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score, balanced_accuracy_score,
    precision_score, recall_score, f1_score,
    roc_auc_score, average_precision_score
)

def build_results(y_true, predictions, scores):
    results = {}
    for name in predictions.keys():
        y_pred = predictions[name]
        y_score = scores.get(name, None)

        cm = confusion_matrix(y_true, y_pred)

        acc = accuracy_score(y_true, y_pred)
        bacc = balanced_accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, pos_label=1, zero_division=0)
        rec  = recall_score(y_true, y_pred, pos_label=1, zero_division=0)
        f1v  = f1_score(y_true, y_pred, pos_label=1, zero_division=0)

        roc_auc = np.nan
        pr_auc  = np.nan
        if y_score is not None:
            roc_auc = roc_auc_score(y_true, y_score)
            pr_auc  = average_precision_score(y_true, y_score)

        results[name] = {
            "confusion_matrix": cm,
            "Accuracy": acc,
            "BalancedAcc": bacc,
            "Precision": prec,
            "Recall": rec,
            "F1": f1v,
            "ROC_AUC": roc_auc,
            "PR_AUC": pr_auc
        }
    return results

results_ethic = build_results(y_test, predictions_ethic, scores_ethic)

list(results_ethic.keys())


## Confusion matrix display (ethical models)

We display confusion matrices for each ethical model to observe:
- False Positives (FP)
- False Negatives (FN)
- True Positives (TP)
- True Negatives (TN)


In [ ]:
# ================================
# 2) Confusion matrices (ethical)
# ================================
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, title):
    plt.figure(figsize=(4.6, 4.2))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = [0, 1]
    plt.xticks(tick_marks, ["0 (Stay)", "1 (Leave)"])
    plt.yticks(tick_marks, ["0 (Stay)", "1 (Leave)"])

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha="center", va="center")

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()
    plt.show()

for name, r in results_ethic.items():
    plot_confusion_matrix(r["confusion_matrix"], f"Confusion Matrix — {name} (Ethical)")


## ROC and Precision-Recall curves (ethical models)

We plot:
- ROC curves (with ROC-AUC)
- Precision-Recall curves (with PR-AUC / Average Precision)

PR curves are recommended for imbalanced classification.


In [ ]:
# ================================
# 3) ROC + PR curves (ethical)
# ================================
from sklearn.metrics import roc_curve, precision_recall_curve

# ROC
plt.figure(figsize=(6,5))
for name in models_ethic.keys():
    y_score = scores_ethic.get(name, None)
    if y_score is None:
        continue
    fpr, tpr, _ = roc_curve(y_test, y_score)
    auc = roc_auc_score(y_test, y_score)
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")

plt.plot([0,1],[0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve — Ethical models")
plt.legend()
plt.tight_layout()
plt.show()

# PR
plt.figure(figsize=(6,5))
for name in models_ethic.keys():
    y_score = scores_ethic.get(name, None)
    if y_score is None:
        continue
    prec, rec, _ = precision_recall_curve(y_test, y_score)
    ap = average_precision_score(y_test, y_score)
    plt.plot(rec, prec, label=f"{name} (AP={ap:.3f})")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve — Ethical models")
plt.legend()
plt.tight_layout()
plt.show()


## Metrics table (ethical models)

We aggregate metrics into a single table and sort by PR-AUC (recommended for imbalance).


In [ ]:
# ================================
# 4) Metrics table (ethical)
# ================================
import pandas as pd

rows = []
for name, r in results_ethic.items():
    rows.append({
        "Model": name,
        "Accuracy": r["Accuracy"],
        "BalancedAccuracy": r["BalancedAcc"],
        "Precision(1)": r["Precision"],
        "Recall(1)": r["Recall"],
        "F1(1)": r["F1"],
        "ROC_AUC": r["ROC_AUC"],
        "PR_AUC": r["PR_AUC"]
    })

metrics_table_ethic = pd.DataFrame(rows).sort_values(by="PR_AUC", ascending=False)
metrics_table_ethic


## Cross-Validation (Ethical Models)

**Principle of Cross-Validation:**
Cross-validation is a statistical method used to estimate the performance of a model on unseen data. It addresses the limitation of a single train-test split, which can be sensitive to how the data is partitioned.

In **k-fold cross-validation**, the dataset is divided into *k* subsets (folds). The model is trained *k* times, each time using *k-1* folds for training and the remaining fold for validation. The final performance metric is the average of the results from all *k* trials. This provides a more robust and reliable estimate of the model's generalization ability.

In [ ]:
from sklearn.model_selection import cross_val_score

print("Evaluating ethical models using 5-fold cross-validation (Accuracy):")
for name, model in models_ethic.items():
    # Using the training set for cross-validation to assess stability
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    print(f"{name:20}: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## Feature Importance (Ethical Models)

**Principle of Feature Importance:**
Feature importance refers to techniques that assign scores to input features based on their contribution to the model's predictive performance. Understanding feature importance is crucial for:
- **Model Interpretability:** Identifying which factors (e.g., salary, work-life balance, distance from home) most heavily influence model predictions.
- **Feature Selection:** Identifying and potentially removing irrelevant or redundant features to simplify the model and improve performance.
- **Business Insights:** Providing actionable insights to stakeholders about the primary drivers of the target outcome (employee attrition).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# We use the RandomForest model as it provides reliable native feature importance scores
if "RandomForest" in models_ethic:
    rf_pipeline = models_ethic["RandomForest"]
    # Ensure the model is fitted (it should be from previous steps)
    rf_model = rf_pipeline.named_steps["model"]
    preprocessor = rf_pipeline.named_steps["preprocess"]

    # Get feature names from the preprocessor
    # This requires scikit-learn >= 1.0
    try:
        feature_names = preprocessor.get_feature_names_out()
    except:
        # Fallback if get_feature_names_out is not available
        feature_names = [f"feature_{i}" for i in range(len(rf_model.feature_importances_))]

    # Get importance values
    importances = rf_model.feature_importances_

    # Create a DataFrame for sorting and plotting
    feat_df = pd.DataFrame({"Feature": feature_names, "Importance": importances})
    feat_df = feat_df.sort_values(by="Importance", ascending=False).head(15)

    # Plotting with the same fancy style as non-ethical
    plt.figure(figsize=(10, 8))
    sns.barplot(x="Importance", y="Feature", data=feat_df, hue="Feature", palette="magma", legend=False)
    plt.title("Top 15 Most Important Features - RandomForest (Ethical)")
    plt.xlabel("Relative Importance Score")
    plt.ylabel("Features")
    plt.grid(axis="x", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()
else:
    print("RandomForest model not found in the ethical models dictionary.")
## -----------------------------------------------------------------------------------

## Final comparison: ethical vs non-ethical

We combine:
- `metrics_table` (non-ethical baseline)
- `metrics_table_ethic` (ethical filtered features)
- Cross-validation results from both approaches

and compare them in a restructured table with ethical/non-ethical side by side.


In [ ]:
# ================================
# 5) Final comparison table (restructured)
# ================================

# First, let's compute cross-validation scores for storage
cv_scores_non_ethical = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    cv_scores_non_ethical[name] = cv_scores.mean()

cv_scores_ethical = {}
for name, model in models_ethic.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    cv_scores_ethical[name] = cv_scores.mean()

# Create restructured comparison table
# We'll pivot the data so models are rows and metrics are columns with ethical/non-ethical sub-columns

# Get unique model names
model_names = list(metrics_table["Model"].unique())

# Create comparison dataframe
comparison_data = []
for model_name in model_names:
    # Get non-ethical metrics
    non_eth = metrics_table[metrics_table["Model"] == model_name].iloc[0]
    # Get ethical metrics
    eth = metrics_table_ethic[metrics_table_ethic["Model"] == model_name].iloc[0]
    
    row = {
        "Model": model_name,
        "Accuracy (Non-Ethical)": non_eth["Accuracy"],
        "Accuracy (Ethical)": eth["Accuracy"],
        "Balanced Accuracy (Non-Ethical)": non_eth["BalancedAccuracy"],
        "Balanced Accuracy (Ethical)": eth["BalancedAccuracy"],
        "Precision (Non-Ethical)": non_eth["Precision(1)"],
        "Precision (Ethical)": eth["Precision(1)"],
        "Recall (Non-Ethical)": non_eth["Recall(1)"],
        "Recall (Ethical)": eth["Recall(1)"],
        "F1 (Non-Ethical)": non_eth["F1(1)"],
        "F1 (Ethical)": eth["F1(1)"],
        "ROC AUC (Non-Ethical)": non_eth["ROC_AUC"],
        "ROC AUC (Ethical)": eth["ROC_AUC"],
        "PR AUC (Non-Ethical)": non_eth["PR_AUC"],
        "PR AUC (Ethical)": eth["PR_AUC"],
        "CV Accuracy (Non-Ethical)": cv_scores_non_ethical.get(model_name, 0),
        "CV Accuracy (Ethical)": cv_scores_ethical.get(model_name, 0)
    }
    comparison_data.append(row)

final_comparison = pd.DataFrame(comparison_data)

# Sort by PR AUC (Ethical) descending
final_comparison = final_comparison.sort_values(by="PR AUC (Ethical)", ascending=False)

print("\n=== Final Comparison: Ethical vs Non-Ethical ===")
print("\nTable shows metrics side-by-side for each model:")
print("Models are rows, parameters are columns with Ethical/Non-Ethical comparison\n")
final_comparison

---------------------------------------------------